# Experiment: _Remove infrequent labels_

Some node labels occur only once in the dataset. This is a test whether they can be removed safely.

Note that some labels can occur only once in the train set, and **multiple** times in the test set! These labels would not get matched.

In [ ]:
from notebook_prelude import *

In [ ]:
NA_VAL = '-'
EXPERIMENT_NAME = 'experiment_remove_infrequent_nodelabels'
df = results_helper.get_experiments_by_names([EXPERIMENT_NAME], fetch_predictions=True)
df_ = results_helper.get_results(filter_out_non_complete_datasets=None, fetch_predictions=True)

df = results_helper.filter_out_datasets(df, lambda x: 'RemoveInfrequentGraphLabels' in x.graph_preprocessing.values)
df = df.append(df_[df_.combined == False]).fillna('-').reset_index()
df = df[(df.combined == False) & (df.kernel != 'unknown')]

## Results when removing single occurrence labels from the dataset

Removing the labels decreases the score significantly!

In [ ]:
df.columns

In [ ]:
df[df.type == TYPE_CONCEPT_MAP].fillna('(na)').groupby(['dataset', 'graph_preprocessing']).prediction_score_f1_macro.max().to_frame().unstack()

In [ ]:
df.loc[df[df.graph_preprocessing == '-'].groupby('dataset').mean_test_f1_macro.idxmax()]
#df.graph_preprocessing.unique()

In [ ]:
best = df.loc[df.fillna('(na)').groupby(['dataset', 'graph_preprocessing']).mean_test_f1_macro.idxmax()]
best[['dataset', 'params', 'mean_test_f1_macro']]